In [3]:
##############################################
# Task subdivision based on reachability     #
##############################################

# Packages
import numpy as np
import rospy
import rospkg
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [4]:
##############################################
# Helper Functions                           #
##############################################


# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T # num vars x num_samples


In [13]:
##############################################
# Create data for testing                    #
##############################################
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


t_surf = np.array([0.43070987363200586, -0.013269419259123479, 0.06385816493811966])
R_surf = np.array([0.008339958880240922, 0.004026424369577458, -0.029073916288389654, 0.9995343617826891])
R_scisurf = ScipyR.from_quat(R_surf)
surf_file = uliconfigpath + '/registration_models/IRC_piece1.csv'

ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('zx',[ang_relative_to_surface,0],degrees=True)


surf = BSplineSurface()
surf.loadSurface(surf_file)


starting = [[0.1, 0.1], [0.3, 0.1]]
ending = [[0.1, 0.9], [0.3, 0.9]]
samps = [5,5]

trajectories = []

for ii in range(0,len(starting)):
    temp_vars = interpMultD(starting[ii],ending[ii],samps[ii])
    data_temp = np.zeros((7,samps[ii]))
    
    for jj in range(0,np.shape(temp_vars)[1]):
        # get surface coordinates
        r, n_hat, r_u_norm, r_v_norm = surf.calculate_surface_point(temp_vars[0,jj],temp_vars[1,jj])
        
        R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))

        # apply surface transform
        q_new = np.round((R_scisurf * R_surf * R_tool_surf).as_quat(),3)
        r_new = R_scisurf.apply(r) + t_surf
        
        data_temp[0:3,jj] = r_new
        data_temp[3:,jj] = q_new
    
    trajectories.append(data_temp)

    
print(trajectories)

[array([[ 0.5209272 ,  0.58551702,  0.65423084,  0.72890654,  0.81241956],
       [ 0.23676384,  0.23338339,  0.22861369,  0.22423746,  0.21902298],
       [ 0.10398773,  0.13165702,  0.15861858,  0.18358378,  0.19105203],
       [-0.012     , -0.004     , -0.002     ,  0.005     ,  0.012     ],
       [-0.219     ,  0.188     ,  0.182     , -0.113     ,  0.036     ],
       [-0.001     ,  0.045     ,  0.033     , -0.023     , -0.019     ],
       [ 0.976     , -0.981     , -0.983     ,  0.993     ,  0.999     ]]), array([[ 0.51652611,  0.58464979,  0.65046391,  0.72450303,  0.80910639],
       [ 0.18498958,  0.18007993,  0.17501205,  0.16988867,  0.1638408 ],
       [ 0.10250383,  0.13170084,  0.15748503,  0.18240393,  0.19015167],
       [-0.008     , -0.009     , -0.001     , -0.006     , -0.011     ],
       [-0.215     ,  0.193     ,  0.183     ,  0.112     , -0.036     ],
       [-0.003     ,  0.045     ,  0.031     ,  0.046     ,  0.041     ],
       [ 0.976     , -0.98      , -

In [14]:
##############################################
# Run Splitting algorithm                    #
##############################################

# input to algorithm is a list of 7xsamps trajectories and a urdf
urdf_file = ''

# TODO: include the reachability stuff
# TODO: create a mask of not done, excluded, done

###############################################
# Step 1: cull based on reachability          #
###############################################